# Финальное задание
### by Egor Makhov. Coursera 2016

In [13]:
import pandas
import numpy as np
import scipy as sp
import time
import datetime

from operator import itemgetter
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt 

---
Загрузка данных

In [6]:
features = pandas.read_csv('./features.csv', index_col='match_id')
features_test = pandas.read_csv('./features_test.csv', index_col='match_id')

1

---
### Подход 1: градиентный бустинг "в лоб"

radiant_win - целевая переменная

In [76]:
y_train = np.ravel(np.array(features[['radiant_win']])) #target feature
features = features.drop(features.loc[:,'duration':'barracks_status_dire'],axis=1)
features.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16


Поиск столбцов с пропусками

In [77]:
for i in xrange(len(features.columns)):
    if (features[[i]].isnull().sum()[0]!=0):
        print features.columns[i]
features = features.fillna(0)

X_train = np.array(features[range(2,len(features.columns))]) # - match_id - start_time

first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time


first_blood_time: За первые 5 минут не было совершено убийств

radiant_courier_time: За первые 5 минут никто не купил курьера

radiant_flying_courier_time: За первые 5 минут никто не улучшил курьера

In [5]:
# Градиентный бустинг с [10,20,30] деревьями
kf = KFold(y_train.size,n_folds=5,shuffle=True)

result = {}
for i in [10,20,30]:
    clf = GradientBoostingClassifier(n_estimators=i)
    clf.fit(X_train,y_train)
    result[i] = np.mean(cross_val_score(clf, X_train,y_train, cv=kf, scoring='roc_auc'))

In [6]:
print result

{10: 0.66446223377751701, 20: 0.68177691030524079, 30: 0.68954600478277883}


In [7]:
# Замер времени + поиск лучших решений
result = {}
for i in xrange(30,36):
    start_time = datetime.datetime.now()
    clf = GradientBoostingClassifier(n_estimators=i)
    clf.fit(X_train,y_train)
    result[i] = np.mean(cross_val_score(clf, X_train,y_train, cv=kf, scoring='roc_auc'))
    print 'Time elapsed:', datetime.datetime.now() - start_time
print result

Time elapsed: 0:03:46.682773
Time elapsed: 0:04:23.848519
Time elapsed: 0:04:59.234498
Time elapsed: 0:04:14.695353
Time elapsed: 0:04:33.539044
Time elapsed: 0:04:18.175560
{32: 0.69057333797368192, 33: 0.69094800701755887, 34: 0.69159500819772479, 35: 0.69220640200144268, 30: 0.68954600478277883, 31: 0.69011361282927464}


---
### Подход 2: логистическая регрессия

In [8]:
# Логистическая со всеми переменными
result = {}
for i in np.arange(0.1,1.0,0.1):
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2',C=i)
    clf.fit(X_train,y_train)
    result[i] = np.mean(cross_val_score(clf, X_train,y_train, cv=kf, scoring='roc_auc'))
    print 'Time elapsed:', datetime.datetime.now() - start_time
print result

Time elapsed: 0:02:27.662202
Time elapsed: 0:02:33.941543
Time elapsed: 0:02:16.197572
Time elapsed: 0:02:15.050582
Time elapsed: 0:02:11.960402
Time elapsed: 0:02:10.612949
Time elapsed: 0:02:23.997805
Time elapsed: 0:02:06.600298
Time elapsed: 0:02:25.794981
{0.5: 0.71644341412194523, 0.20000000000000001: 0.71639411146383236, 0.40000000000000002: 0.71641434968743944, 0.80000000000000004: 0.71648084508444077, 0.30000000000000004: 0.71642757606099639, 0.59999999999999998: 0.71646239584765192, 0.10000000000000001: 0.71640000383094704, 0.90000000000000002: 0.71644129611923568, 0.70000000000000007: 0.71640885460231929}


In [78]:
# Удаляем категоральные признаки
heroes = features.loc[:,['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                         'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']]
features = features.drop(['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                         'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)
X_train = np.array(features[range(2,len(features.columns))])
features.head()

,start_time,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,r2_level,r2_xp,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,5,2098,1489,20,0,0,7,3,842,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,4,1188,1033,9,0,1,12,4,1596,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,4,1319,1270,22,0,0,12,3,1314,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,4,1779,1056,14,0,0,5,2,539,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,4,1431,1090,8,1,0,8,2,629,...,0,46,182,-80,225,6,3,3,0,-16


In [11]:
# Логистическая без категоральных
result = {}
for i in np.arange(0.1,1.0,0.1):
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2',C=i)
    clf.fit(X_train,y_train)
    result[i] = np.mean(cross_val_score(clf, X_train,y_train, cv=kf, scoring='roc_auc'))
    print 'Time elapsed:', datetime.datetime.now() - start_time
print result

Time elapsed: 0:01:46.465352
Time elapsed: 0:01:55.342489
Time elapsed: 0:01:41.572286
Time elapsed: 0:01:29.400960
Time elapsed: 0:01:48.169154
Time elapsed: 0:01:52.525588
Time elapsed: 0:01:54.193830
Time elapsed: 0:01:56.582918
Time elapsed: 0:01:41.283717
{0.5: 0.71641256252893259, 0.20000000000000001: 0.71646023621715382, 0.40000000000000002: 0.71649828752840716, 0.80000000000000004: 0.71646700986675893, 0.30000000000000004: 0.71648344231841243, 0.59999999999999998: 0.71644646400194811, 0.10000000000000001: 0.71649338960799924, 0.90000000000000002: 0.71649879459942745, 0.70000000000000007: 0.71644133487270278}


In [79]:
# Создаем мешок слов по героям
uni_heroes = pandas.Series(heroes.values.ravel()).unique()

N = len(uni_heroes) # 108
X_pick = np.zeros((heroes.shape[0], N))

for i, match_id in enumerate(heroes.index):
    for p in xrange(5):
        X_pick[i, np.where(uni_heroes == heroes.ix[match_id, 'r%d_hero' % (p+1)])[0][0]] = 1
        X_pick[i, np.where(uni_heroes == heroes.ix[match_id, 'd%d_hero' % (p+1)])[0][0]] = -1

In [80]:
X_train = np.hstack([X_train,X_pick])

In [69]:
# Логистическая с мешком слов
result = {}
for i in np.arange(0.1,1.0,0.1):
    start_time = datetime.datetime.now()
    clf = LogisticRegression(penalty='l2',C=i)
    clf.fit(X_train,y_train)
    result[i] = np.mean(cross_val_score(clf, X_train,y_train, cv=kf, scoring='roc_auc'))
    print 'Time elapsed:', datetime.datetime.now() - start_time
print result

Time elapsed: 0:04:26.994628
Time elapsed: 0:04:38.102621
Time elapsed: 0:05:19.696014
Time elapsed: 0:06:12.722243
Time elapsed: 0:05:43.551478
Time elapsed: 0:06:07.060200
Time elapsed: 0:05:19.445010
Time elapsed: 0:05:11.522042
Time elapsed: 0:06:10.405744
{0.5: 0.75176149182344898, 0.20000000000000001: 0.75177082281883456, 0.40000000000000002: 0.75178583833052648, 0.80000000000000004: 0.75181176198153099, 0.30000000000000004: 0.75175626958299502, 0.59999999999999998: 0.75177740248573866, 0.10000000000000001: 0.75177716098027825, 0.90000000000000002: 0.7517692855867758, 0.70000000000000007: 0.75177644953867984}


In [85]:
# Best model training
clf = LogisticRegression(penalty='l2',C=0.8)
clf.fit(X_train,y_train)

LogisticRegression(C=0.8, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [100]:
# Test data preprocessing
features_test = features_test.fillna(0)
heroes_test = features_test.loc[:,['r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                                   'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero']]
features_test = features_test.drop(['lobby_type','r1_hero','r2_hero','r3_hero','r4_hero','r5_hero',
                         'd1_hero','d2_hero','d3_hero','d4_hero','d5_hero'],axis=1)

X_pick = np.zeros((heroes_test.shape[0], N))

for i, match_id in enumerate(heroes_test.index):
    for p in xrange(5):
        X_pick[i, np.where(uni_heroes == heroes_test.ix[match_id, 'r%d_hero' % (p+1)])[0][0]] = 1
        X_pick[i, np.where(uni_heroes == heroes_test.ix[match_id, 'd%d_hero' % (p+1)])[0][0]] = -1
X_test = np.array(features_test[range(2,len(features_test.columns))])
X_test = np.hstack([X_test,X_pick])

In [101]:
# Final result
print clf.predict_proba(X_test)[:, 1].max()
print clf.predict_proba(X_test)[:, 1].min()

0.996637288586
0.00829230830628


---
### Ответы 

Вопрос 1 
first_blood_time
first_blood_team
first_blood_player1
first_blood_player2
radiant_bottle_time
radiant_courier_time
radiant_flying_courier_time
radiant_first_ward_time
dire_bottle_time
dire_courier_time
dire_flying_courier_time
dire_first_ward_time

first_blood_time: За первые 5 минут не было совершено убийств
radiant_courier_time: За первые 5 минут никто не купил курьера

Вопрос 2
radiant_win - целевая переменная

Вопрос 3
Time elapsed: 0:03:46.682773
30: 0.68954600478277883

Вопрос 4
Не имеет, так как качество растет не особо быстро и возникает шанс переобучения.
Чтобы ускорить можно уменьшить количество входных данных (брать только экстремумы), а так же настраивать распределение задач по ядрам процессора

Вопрос 1
0.71648084508444077 при С=0.8
Качество заметно лучше по сравнению с градиентным бустингом. Это объясняется тем, что градиентный логистическая регрессия более эффективна как класс алгоритмов для такого типа задач. При этом не стоит забывать, что градиентный бустинг легче настраивать.
Time elapsed: 0:02:06.600298 
Следовательно, логистическая регрессия на тех же данных работает быстрее.

Вопрос 2.
Качество почти не изменилось. Это объясняется тем, что логистическая регрессия сама по себе игнорирует категориальные признаки, давая и малый вес в общей модели. Однако, скорость подсчета увеличилась из-за общего снижения кол-ва данных.

Вопрос 3.
108 различных идентификаторов героев.
По крайней мере в предоставленных данных.

Вопрос 4.
0.75181176198153099
Качество улучшилось. Это объясняется принципом работы логистической регрессии. Мы разбили на 108 признаков и у каждого есть свой вес, теперь наши бывшие категориальные признаки оказывают большее влияние на предсказание.
Стоит заметить, что скорость заметно снизилась.
Time elapsed: 0:05:11.522042

Вопрос 5.
макс 0.996637288586
мин 0.00829230830628